In [4]:
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

def is_number(s):
    try:
        float(s)
        return True 
    except (ValueError,AttributeError):
        return False 
    
# Main script
root_dir = '/glade/u/home/hongli/work/sharp/basins/tayprk'
output_dir=os.path.join(root_dir,'analysis/step4_plot_obj_var_trace')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

sim_file='route.nc'
# q_vname1 = 'KWTroutedRunoff'
q_vname2 = 'IRFroutedRunoff'
t_vname = 'time'
time_format='%Y-%m-%d'
obs_file='/glade/u/home/hongli/work/sharp/basins/tayprk/analysis/Taylor_park_09107000.txt'

trial_folder_names=['calib_trial5','calib_trial6','calib_trial7','calib_trial8','calib_trial9','calib_trial10']
param_nums=[10, 10, 10, 10, 10, 10]
job_nums=[4, 4, 4, 4, 4, 4, 4, 4]
itr_per_job = 150
obj_names = ['RMSE', 'RMSE', 'RMSE', 'NegKGE', 'NegKGE', 'NegKGE']

ostIn_file = 'ostIn.txt'
param_tpl = 'nc_multiplier.tpl'
trialParam_file = 'trialParams.model_huc12_3hr.v1.nc_pattern'

for i in range(len(trial_folder_names)):
    
    trial_folder_name = trial_folder_names[i]
    param_num = param_nums[i]
    job_num = job_nums[i]    
    outputfile = trial_folder_name+'.png'
    obj_name = obj_names[i]
        
    print(trial_folder_name)

    # read var ranges from ostIn.txt
    param_range = []
    with open(os.path.join(root_dir,trial_folder_name,ostIn_file), 'r+') as f:
        for line_counter, line in enumerate(f):
            line = line.strip()
            if line and line.startswith('BeginParams'):
                line_param_value_start = line_counter+2
            elif line and line.startswith('EndParams'):
                line_param_value_end = line_counter-1
                break

    with open(os.path.join(root_dir,trial_folder_name,ostIn_file), 'r+') as f:
        for line_counter, line in enumerate(f):
            line = line.strip()
            if line and line_counter >= line_param_value_start and line_counter <= line_param_value_end:
                param_range.append([float(s) for s in line.split()[1:1+3]]) # init, min, max
            elif line_counter > line_param_value_end:
                break
    param_range=np.asarray(param_range)
    param_range[-2,:]=[0.01,0.01,5] # heightCanopyBottom
    param_range[-1,:]=[1,0.1,20] # heightCanopyTop
    
    # read parameters corresponding variable names
    var_names=[]
    with open (os.path.join(root_dir,trial_folder_name,param_tpl), 'r') as f:
        for line in f:
            line=line.strip()
            if line and not line.startswith('!') and not line.startswith("'"):
                splits=line.split('|')
                if isinstance(splits[1].strip(), str):
                    var_names.append(splits[0].strip())    
    
    # read variable initial values from trialParam.nc
    var_init = []
    f = xr.open_dataset(os.path.join(root_dir,trial_folder_name,trialParam_file))
    for var_name in var_names:
        var_init.append(f[var_name].values[:][0]) 
    var_init=np.asarray(var_init)
        
    # calculate var range
    var_range = np.zeros_like(param_range)
    for k in range(param_num-2):
        var_range[k,:] = np.multiply(var_init[k],param_range[k,:])
    var_range[-2:,:] = param_range[-2:,:]
    
    # read OstOutput.txt or calib records
    solution_cum_ids=[]
    objs = []
    param_set_values = []
    
    record_dir = os.path.join(root_dir, trial_folder_name)
    record_keyword = trial_folder_name.split('_')[1]
    record_files = [d for d in os.listdir(record_dir) if record_keyword in d]
    record_files = sorted(record_files)

    for j in range(len(record_files)):
        record_file = record_files[j]

        with open(os.path.join(root_dir,trial_folder_name,record_file), 'r+') as f:
            for line_counter, line in enumerate(f):
                line = line.strip()
                if line and line.startswith('Ostrich Run Record'):
                    line_param_name = line_counter+1
                    line_param_value_start = line_counter+2
                elif line and line.startswith('Optimal Parameter Set'):
                    line_param_value_end = line_counter-2
                    break

        with open(os.path.join(root_dir,trial_folder_name,record_file), 'r+') as f:
            for line_counter, line in enumerate(f):
                line = line.strip()
                if line and line_counter == line_param_value_start-1:
                    param_names = line.split()[3:-2]
                elif line and line_counter >= line_param_value_start and line_counter <= line_param_value_end:  
                    
                    if all([is_number(s) for s in line.split()]) == True:

                        solution_id = int(line.split()[0]) 
                        solution_cum_id = solution_id + j*itr_per_job
                        solution_cum_ids.append(solution_cum_id)

                        objs.append(float(line.split()[1]))
                        param_set_values.append([float(x) for x in line.split()[2:-1]])
                    
                elif line_counter > line_param_value_end:
                    break
    param_set_values=np.asarray(param_set_values)
        
    # calculate heightCanopyBottom and heightCanopyTop    
    h_canopy_btm = var_init[-2]*param_set_values[:,-2] # Bottom = Bottom_initial*Bottom_multiplier
    h_canopy_top = h_canopy_btm + np.multiply(param_set_values[:,-1],(param_range[-1,-1]-h_canopy_btm)) # Top = Bottom + Top_ip*(20 – Bottom)

    # plot
    alpha = 'a'
    row_num=4
    col_num=3        
    fig, ax = plt.subplots(row_num,col_num)
    fig.set_figwidth(3.0*col_num) #90mm
    fig.set_figheight(3.5*0.5*row_num) 
    dpi_value=150

    for i in range(row_num):
        for j in range(col_num):
            if i==(row_num-1) and j>=(col_num-2): # blank subplot
                ax[i,j].axis('off')
            else:
                param_id = i*col_num + j - 1
                if i==0 and j==0: # obj function
                    ax[i,j].plot(solution_cum_ids, objs, 'k-o', linewidth=0.75, markersize=1.0)
                    fig_count = i*col_num + j
                    title_str = '('+chr(ord(alpha) + fig_count) +') ' + 'Objective function'   
                    y_label = obj_name 
                elif i==(row_num-1) and j==0: # heightCanopy
                    ax[i,j].plot(solution_cum_ids, h_canopy_btm, 'b-o', linewidth=0.75, markersize=1.0, label='Bottom')
                    ax[i,j].plot(solution_cum_ids, h_canopy_top, 'r-^', linewidth=0.75, markersize=1.0, label='Top')

                    fig_count = i*col_num + j
                    title_str = '('+chr(ord(alpha) + fig_count) +') ' + 'heightCanopy'
                    y_label = 'heightCanopy'
#                     ax[i,j].set_ylim(param_range[param_id,1],param_range[param_id+1,2])
#                     ax[i,j].set_ylim(0,5)
                    ax[i,j].legend(loc='upper right',fontsize='x-small')
                else: # other parameters
                    ax[i,j].plot(solution_cum_ids, var_init[param_id]*param_set_values[:, param_id], 'k-o', linewidth=0.75, markersize=1.0)

                    fig_count = i*col_num + j
                    title_str = '('+chr(ord(alpha) + fig_count) +') ' + (param_names[param_id]).replace('_mtp', '')
                    y_label = var_names[param_id]
                    ax[i,j].set_ylim(var_range[param_id,1],var_range[param_id,2])

                #axis, label, title, legend
                ax[i,j].set_title(title_str, fontsize='small', fontweight='semibold')
                ax[i,j].set_xlabel('Iterations', fontsize='small')
                ax[i,j].set_ylabel(y_label, fontsize='small')

    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small')                
    fig.tight_layout()
    fig.savefig(os.path.join(output_dir, outputfile), dpi=dpi_value)
    plt.close(fig)      
    del fig

print('Done')

calib_trial5
calib_trial6
calib_trial7
calib_trial8
calib_trial9
calib_trial10
Done


In [31]:
h_canopy_btm

array([0.1      , 2.79044  , 2.723502 , 2.723502 , 0.8094672, 1.326884 ,
       1.156234 , 1.156234 , 1.156234 , 1.212913 , 1.212913 , 1.212913 ,
       1.212913 , 1.212913 , 1.212913 , 1.212913 , 1.212913 , 1.212913 ,
       1.212913 , 1.212913 , 1.212913 , 1.208892 , 1.208892 , 1.208892 ,
       1.208892 , 1.208892 , 1.208892 , 1.208892 , 1.208892 , 1.208892 ,
       1.208892 , 1.208892 , 1.208892 , 1.208892 ])

In [32]:
h_canopy_top

array([0.9999974 , 3.56875956, 3.5048489 , 3.5048489 , 0.87076786,
       1.38653187, 1.21642698, 1.21642698, 1.21642698, 1.27292493,
       1.27292493, 1.27292493, 1.27292493, 1.27292493, 1.27292493,
       1.27292493, 1.27292493, 1.27292493, 1.27292493, 1.27292493,
       1.27292493, 1.26891677, 1.26891677, 1.26891677, 1.26891677,
       1.26891677, 1.26891677, 1.26891677, 1.26891677, 1.26891677,
       1.26891677, 1.26891677, 1.26891677, 1.26891677])

In [33]:
param_set_values[:,-1]

array([0.045226  , 0.045226  , 0.045226  , 0.045226  , 0.00319432,
       0.00319432, 0.00319432, 0.00319432, 0.00319432, 0.00319432,
       0.00319432, 0.00319432, 0.00319432, 0.00319432, 0.00319432,
       0.00319432, 0.00319432, 0.00319432, 0.00319432, 0.00319432,
       0.00319432, 0.00319432, 0.00319432, 0.00319432, 0.00319432,
       0.00319432, 0.00319432, 0.00319432, 0.00319432, 0.00319432,
       0.00319432, 0.00319432, 0.00319432, 0.00319432])